## GPT2

In [26]:
import pandas as pd
from transformers import pipeline, set_seed

# Configuration pour assurer la reproductibilité lors de la génération de texte
set_seed(123)

data = {
    'business_id': ['123', '123', '456', '456', '456'],
    'text': [
        "Great service but the food was bland.",
        "Loved the ambiance and the dessert, but the main course was too salty.",
        "The waiter was rude.",
        "The steak was fantastic, but the room was too noisy.",
        "Amazing cocktails and friendly staff, but the price is too high."
    ]
}

df = pd.DataFrame(data)


In [29]:
# Initialiser le modèle GPT-2 
generator = pipeline('question-answering', model='gpt2')

def generate_insights(reviews):
    combined_reviews = " ".join(reviews)
    prompt = (f"Here are some customer reviews on a restaurant deduct the restaurant's asset's.\n\n{combined_reviews}")
    
    summary = generator(prompt,max_length=250, num_return_sequences=1)
    return summary[0]['generated_text']

# Grouper les avis par 'business_id'
grouped_reviews = df.groupby('business_id')['text'].apply(list)

# Générer des insights pour chaque groupe d'avis
insights_by_business = grouped_reviews.apply(generate_insights)

# Afficher les insights
for business_id, insights in insights_by_business.items():
    print(f"Business ID: {business_id}\nInsights:\n{insights}\n")


Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at gpt2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: H argument needs to be of type (SquadExample, dict)

In [ ]:
def generate_insights(reviews):
    combined_reviews = " ".join(reviews[:5])  # Limiter le nombre d'avis
    prompt = (f"Based on the following customer reviews, identify the key strengths and weaknesses of the restaurant:\n\n{combined_reviews}\n\nStrengths:\n- Weaknesses:")
    
    summary = generator(prompt, max_length=250, num_return_sequences=1)
    return summary[0]['generated_text']

# Grouper les avis par 'business_id'
grouped_reviews = df.groupby('business_id')['text'].apply(list)

# Générer des insights pour chaque groupe d'avis
insights_by_business = grouped_reviews.apply(generate_insights)

# Afficher les insights
for business_id, insights in insights_by_business.items():
    print(f"Business ID: {business_id}\nInsights:\n{insights}\n")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Business ID: 123
Insights:
Based on the following customer reviews, identify the key strengths and weaknesses of the restaurant:

Great service but the food was bland. Loved the ambiance and the dessert, but the main course was too salty.

Strengths:
- Weaknesses:

- Bad food presentation.

- Complaints of not always making the necessary adjustments. No experience and bad service!

- Overdelivering too many food items.

- Unnecessary server work

- Food is inconsistent.

Cleaning out the restaurant is very important

Overall:

It did not appear overly busy at all.

Overall Rating:

The restaurant was excellent for my needs

Conclusion

This restaurant was excellent for my needs. I ordered the salad salad and had it by my desk next to my car. The salad salad had the perfect amount of sweetness and savoriness to go with it so they served the salad with the right amount of salt. I ordered the fried chicken and it was good. The hot chicken was too salty and fried. The fried chicken looked 